In [1]:
# coding: utf8
import torch
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

model_path = "vinai/PhoGPT-4B-Chat"  
# model_path = "/home/andv/important/Chatbot/finetune_llm/phogpt-mergerd-with-config-1"

# model_path = "/home/andv/important/Chatbot/finetune"

config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)  
# config.init_device = "cuda"
config.attn_config['attn_impl'] = 'flash' # If installed: this will use either Flash Attention V1 or V2 depending on what is installed

model = AutoModelForCausalLM.from_pretrained(model_path,
                                             config=config, 
                                             torch_dtype=torch.bfloat16, 
                                             trust_remote_code=True).to("cuda")

# model = AutoModelForCausalLM.from_pretrained(model_path, config = config, trust_remote_code=True).to("cuda")
# If your GPU does not support bfloat16:
# model = AutoModelForCausalLM.from_pretrained(model_path, config=config, torch_dtype=torch.float16, trust_remote_code=True)
model.eval()  

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)  

PROMPT_TEMPLATE = "### Câu hỏi: {instruction}\n### Trả lời:"  
# PROMPT_TEMPLATE = "{instruction}"  
# Some instruction examples
# instruction = "Viết bài văn nghị luận xã hội về {topic}"
# instruction = "Viết bản mô tả công việc cho vị trí {job_title}"
# instruction = "Sửa lỗi chính tả:\n{sentence_or_paragraph}"
# instruction = "Dựa vào văn bản sau đây:\n{text}\nHãy trả lời câu hỏi: {question}"
# instruction = "Tóm tắt văn bản:\n{text}"



/home/andv/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/f56fc6d71f147a3a293fdab56676337dc6f641e1/configuration_mpt.py:114: UserWarning: alibi or rope is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi or rope is turned on, setting `learned_pos_emb` to `False.`')
/home/andv/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/f56fc6d71f147a3a293fdab56676337dc6f641e1/configuration_mpt.py:141: UserWarning: If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".
  warnings.warn(UserWarning('If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".'))
The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.


ValueError: MPTForCausalLM does not support Flash Attention 2.0 yet. Please request to add support where the model is hosted, on its model hub page: https://huggingface.co/vinai/PhoGPT-4B-Chat/discussions/new or in the Transformers GitHub repo: https://github.com/huggingface/transformers/issues/new

In [2]:
instruction = "Phần mềm quản lý ngân sách và tài chính xã Tuân thủ các quy định nào do Bộ Tài chính ban hành? "
# instruction = "Sửa lỗi chính tả:\nTriệt phá băng nhóm kướp ô tô, sử dụng \"vũ khí nóng\""

input_prompt = PROMPT_TEMPLATE.format_map({"instruction": instruction})  

input_ids = tokenizer(input_prompt, return_tensors="pt")  

outputs = model.generate(  
    inputs=input_ids["input_ids"].to("cuda"),  
    attention_mask=input_ids["attention_mask"].to("cuda"),  
    do_sample=True,  
    temperature=1.0,  
    top_k=50,  
    top_p=0.9,  
    max_new_tokens=1024,  
    eos_token_id=tokenizer.eos_token_id,  
    pad_token_id=tokenizer.pad_token_id  
)  

response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  
# response = tokenizer.decode(outputs[0])
response = response.split("### Trả lời:")[1]

/home/andv/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/f56fc6d71f147a3a293fdab56676337dc6f641e1/configuration_mpt.py:141: UserWarning: If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".
  warnings.warn(UserWarning('If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".'))


In [3]:
print(response)

Theo Điều 8 Thông tư 19/2020/TT-BTC thì phần mềm quản lý ngân sách và tài chính xã đáp ứng các yêu cầu sau đây:

1. Thực hiện các chức năng quy định tại điểm c khoản 2 Điều 3 Thông tư 19/2020/TT-BTC, trong đó có chức năng nhập dự toán thu, chi ngân sách xã (ngân sách xã không có tài khoản dự toán tại Kho bạc Nhà nước), nhập tình hình thực hiện dự toán thu, chi ngân sách xã.

2. Hỗ trợ thực hiện phân bổ dự toán ngân sách xã cho các đơn vị sử dụng ngân sách trực thuộc (các xã); đồng thời, gửi Ủy ban nhân dân cấp huyện để theo dõi, quản lý theo quy định.

3. Hỗ trợ thực hiện tổng hợp báo cáo thu, chi ngân sách xã (bao gồm cả báo cáo thuyết minh, giải trình kết quả thu và chi ngân sách xã) theo mẫu biểu báo cáo tài chính quy định, gửi Ủy ban nhân dân cấp huyện và cơ quan tài chính cấp huyện.

4. Hỗ trợ thực hiện đối chiếu, xác nhận số liệu thu, chi ngân sách xã với Kho bạc Nhà nước cấp huyện.

5. Hỗ trợ công khai thông tin ngân sách xã và các hoạt động tài chính khác của xã theo quy định c

In [5]:
print(response)

Phần mềm kế toán hành chính là một công cụ phần mềm được sử dụng để quản lý tài chính của tổ chức, doanh nghiệp hoặc cơ quan nhà nước. Phần mềm này cho phép người sử dụng nhập các chi tiết liên quan đến tài chính của tổ chức, bao gồm các giao dịch tài chính như bán hàng, thu chi, tiền lương và tài sản cố định. Phần mềm kế toán hành chính giúp cho người dùng quản lý các hoạt động tài chính của tổ chức một cách nhanh chóng và hiệu quả, giảm thiểu các sai sót và nhầm lẫn trong việc theo dõi và tính toán tài chính. Ngoài ra, phần mềm này cũng giúp tăng cường tính chính xác của các hoạt động tài chính của tổ chức, giúp cho việc quản lý và điều hành tổ chức một cách dễ dàng hơn.


In [4]:
# messages = [
#     {"role": "user", "content": "Kể tên một môn thể thao mạo hiểm"},
#     {"role": "assistant", "content": "Nhảy Bungee."},
#     {"role": "user", "content": "Bạn đã bao giờ đi nhảy bungee chưa"}
# ]

# # Using apply_chat_template
# tokenizer = AutoTokenizer.from_pretrained("vinai/PhoGPT-4B-Chat", trust_remote_code=True)
# input_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [10]:
# input_prompt

'### Câu hỏi: Kể tên một môn thể thao mạo hiểm\n### Trả lời: Nhảy Bungee.</s>\n### Câu hỏi: Bạn đã bao giờ đi nhảy bungee chưa\n### Trả lời:'

In [8]:
# input_ids = tokenizer(input_prompt, return_tensors="pt")  

# outputs = model.generate(  
#     inputs=input_ids["input_ids"].to("cuda"),  
#     attention_mask=input_ids["attention_mask"].to("cuda"),  
#     do_sample=True,  
#     temperature=1.0,  
#     top_k=50,  
#     top_p=0.9,  
#     max_new_tokens=1024,  
#     eos_token_id=tokenizer.eos_token_id,  
#     pad_token_id=tokenizer.pad_token_id  
# )  

# response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  
# # response = tokenizer.decode(outputs[0])
# # response = response.split("### Trả lời:")[1]

In [9]:
response

'### Câu hỏi: Kể tên một môn thể thao mạo hiểm\n### Trả lời: Nhảy Bungee.\n### Câu hỏi: Bạn đã bao giờ đi nhảy bungee chưa\n### Trả lời: Là một mô hình ngôn ngữ AI, tôi không có khả năng tham gia vào các hoạt động thể chất, nên tôi không có kinh nghiệm nhảy bungee.'

In [5]:
input_prompt

'### Câu hỏi: Kể tên một môn thể thao mạo hiểm\n### Trả lời: Nhảy Bungee.</s>\n### Câu hỏi: Bạn đã bao giờ đi nhảy bungee chưa\n### Trả lời:'

In [14]:
response

'Món ăn ngon nhất ở Hà Nội\nĂn gì ở Hà Nội là câu hỏi được đặt ra cho rất nhiều du khách. Bởi Hà Nội là một trong những địa danh nổi tiếng với nhiều món ăn hấp dẫn. Hà Nội được coi là một thiên đường ẩm thực với rất nhiều món ăn ngon, độc đáo. Vậy ăn gì ở Hà Nội?\nBài viết dưới đây sẽ giới thiệu cho bạn top 10 món ăn ngon nhất ở Hà Nội.\nTop 10 món ăn ngon nhất ở Hà Nội\n1. Phở Hà Nội - Một trong những món ăn ngon nhất ở Hà Nội\nPhở là món ăn ngon và nổi tiếng nhất ở Hà Nội. Phở là món ăn truyền thống, được yêu thích từ lâu đời của người dân thủ đô. Phở Hà Nội có hương vị đậm đà, nước dùng thơm ngon, bánh phở mềm dai, kết hợp với các loại gia vị, thảo mộc như hành lá, rau thơm, nước mắm, nước cốt chanh, ớt, hạt tiêu... Phở Hà Nội thường được phục vụ trong các quán phở truyền thống và nổi tiếng nhất là Phở Bát Đàn, Phở Thìn, Phở Lý Quốc Sư.\n2. Bún chả Hà Nội - Một trong những món ăn ngon nhất ở Hà Nội\nBún chả là món ăn ngon và nổi tiếng của Việt Nam. Bún chả Hà Nội được làm từ thịt lợ

In [12]:
response

'Món ăn ngon nhất ở Hà Nội\nKhông chỉ nổi tiếng với những danh lam thắng cảnh đẹp, những món ăn ngon cũng là một điểm nhấn khiến Hà Nội luôn là lựa chọn hàng đầu của du khách trong và ngoài nước.\nMột số món ăn ngon không thể không nhắc đến khi đến Hà Nội:\n- Phở: Đây là món ăn truyền thống được rất nhiều người yêu thích. Phở có thể ăn sáng, trưa, chiều tối đều ngon. Đặc biệt, đến Hà Nội vào mùa đông mà không thưởng thức phở thì thật là đáng tiếc.\n- Bún chả: Đây là món ăn nổi tiếng của Hà Nội và cũng là món ăn ngon được nhiều người nước ngoài biết đến. Bún chả được bán tại các quán ăn ven đường, bạn có thể ghé vào để thưởng thức.\n- Chả cá Lã Vọng: Đây là món ăn ngon được chế biến từ cá lăng, được làm thành chả cá, đem nướng và ăn kèm với bún và các loại rau thơm.\n- Bánh tôm: Đây là món ăn ngon đặc trưng của Hà Nội, được bán nhiều tại các nhà hàng, quán ăn ven đường. Bạn có thể đến các địa điểm như: đường Lý Quốc Sư, Thanh Niên để thưởng thức món ăn ngon này.\n- Chả rươi: Nếu muốn th